<a href="https://colab.research.google.com/github/Ntorres95/nfl_big_data_bowl_2021/blob/main/DataCleaning_and_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3
!pip install fsspec
!pip install s3fs

  Using cached https://files.pythonhosted.org/packages/ef/d5/c0c33ca15e31062220ac5964f3492409eaf90a5cf5399503cd8264f2f8e9/botocore-1.19.25-py2.py3-none-any.whl
ERROR: aiobotocore 1.1.2 has requirement botocore<1.17.45,>=1.17.44, but you'll have botocore 1.19.25 which is incompatible.
  Found existing installation: botocore 1.17.44
    Uninstalling botocore-1.17.44:
      Successfully uninstalled botocore-1.17.44
  Using cached https://files.pythonhosted.org/packages/1e/6a/b6490235c01c941a24a86235e2a641e9505cf0ce4b4968d4987573d92bec/botocore-1.17.44-py2.py3-none-any.whl
ERROR: boto3 1.16.25 has requirement botocore<1.20.0,>=1.19.25, but you'll have botocore 1.17.44 which is incompatible.
  Found existing installation: botocore 1.19.25
    Uninstalling botocore-1.19.25:
      Successfully uninstalled botocore-1.19.25


In [2]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import boto3
import botocore
import os

In [3]:
from boto3.session import Session
from botocore.client import Config
from botocore.handlers import set_list_objects_encoding_type_url

In [4]:
#load aws buckets into pandas
weeks_df = pd.read_csv("https://mybigbowlbucket.s3-us-west-1.amazonaws.com/weeksNFL.csv")
games_df = pd.read_csv("https://mybigbowlbucket.s3-us-west-1.amazonaws.com/games.csv")
players_df = pd.read_csv("https://mybigbowlbucket.s3-us-west-1.amazonaws.com/players.csv")
plays_df = pd.read_csv("https://mybigbowlbucket.s3-us-west-1.amazonaws.com/plays.csv")

In [5]:
weeks_df.shape

(18309388, 19)

In [6]:
weeks_df = weeks_df.dropna()
games_df = games_df.dropna()
players_df = players_df.dropna()
plays_df = plays_df.dropna()

In [7]:
weeks_df.shape

(5117955, 19)

In [8]:
weeks_df.head(3)

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
2,2018-09-07T01:07:14.599Z,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,2018090600,75,left,HITCH
4,2018-09-07T01:07:14.599Z,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,2018090600,75,left,HITCH
10,2018-09-07T01:07:14.599Z,91.29,21.83,0.01,0.02,0.01,250.34,151.83,None,2555415.0,Austin Hooper,81.0,TE,1,away,2018090600,75,left,OUT


In [9]:
games_df.head(3)

,gameId,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,week
0,2018090600,09/06/2018,20:20:00,PHI,ATL,1
1,2018090901,09/09/2018,13:00:00,CLE,PIT,1
2,2018090902,09/09/2018,13:00:00,IND,CIN,1


In [10]:
players_df.head(3)

,nflId,height,weight,birthDate,collegeName,position,displayName
0,2539334,72,190,1990-09-10,Washington,CB,Desmond Trufant
1,2539653,70,186,1988-11-01,Southeastern Louisiana,CB,Robert Alford
2,2543850,69,186,1991-12-18,Purdue,SS,Ricardo Allen


In [11]:
plays_df.head(3)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
29,2018090600,1680,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.299050,False
45,2018090600,2503,(11:04) (Shotgun) M.Ryan pass incomplete short...,3,2,10,ATL,play_type_pass,ATL,32,EMPTY,"1 RB, 1 TE, 3 WR",5.0,4.0,"4 DL, 1 LB, 6 DB",TRADITIONAL,6.0,3.0,11:04:00,42.0,ING,ATL 2,I,0,-12,-1.868122,False
91,2018090901,439,(8:05) (Shotgun) B.Roethlisberger pass short r...,1,3,9,PIT,play_type_pass,PIT,43,EMPTY,"1 RB, 1 TE, 3 WR",6.0,4.0,"2 DL, 4 LB, 5 DB",TRADITIONAL,0.0,0.0,08:05:00,53.0,IBW,PIT 11,C,0,0,-1.770704,False


In [12]:
df = weeks_df.merge(plays_df, how='right', on='playId')
df.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId_x,playId,playDirection,route,gameId_y,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018-09-07T02:17:33.400Z,110.35,32.48,0.00,0.00,0.00,44.48,307.08,None,2506467.0,Darren Sproles,43.0,RB,1.0,home,2.018091e+09,1680,left,WHEEL,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False
1,2018-09-07T02:17:33.400Z,106.49,42.95,0.00,0.00,0.00,247.94,189.89,None,2507763.0,Mike Wallace,14.0,WR,1.0,home,2.018091e+09,1680,left,POST,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False
2,2018-09-07T02:17:33.400Z,105.99,22.35,0.00,0.00,0.00,255.39,246.65,None,2540158.0,Zach Ertz,86.0,TE,1.0,home,2.018091e+09,1680,left,POST,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False
3,2018-09-07T02:17:33.400Z,107.67,16.05,5.83,2.36,0.59,327.16,2.02,None,2552600.0,Nelson Agholor,13.0,WR,1.0,home,2.018091e+09,1680,left,CROSS,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False
4,2018-09-07T02:17:33.400Z,106.75,15.20,0.00,0.00,0.00,253.47,288.27,None,2553502.0,DeAndre Carter,16.0,WR,1.0,home,2.018091e+09,1680,left,POST,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False


In [13]:
df = df.drop(columns=['time', 'x', 'y', 's', 'a', 'o', 'dir', 'event', 'nflId', 'displayName', 'jerseyNumber', 'frameId', 'gameId_x', 'gameId_y', 'playDescription','possessionTeam', 'playType', 'yardlineSide', 'gameClock', 'penaltyCodes', 'penaltyJerseyNumbers','typeDropback'])

In [14]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

df = pd.get_dummies(df)

df.head()

,dis,playId,quarter,down,yardsToGo,yardlineNumber,defendersInTheBox,numberOfPassRushers,preSnapVisitorScore,preSnapHomeScore,absoluteYardlineNumber,offensePlayResult,playResult,epa,isDefensivePI,position_FB,position_HB,position_QB,position_RB,position_TE,position_WR,team_away,team_home,playDirection_left,playDirection_right,route_ANGLE,route_CORNER,route_CROSS,route_FLAT,route_GO,route_HITCH,route_IN,route_OUT,route_POST,route_SCREEN,route_SLANT,route_WHEEL,route_undefined,offenseFormation_EMPTY,offenseFormation_I_FORM,...,offenseFormation_WILDCAT,"personnelO_0 RB, 0 TE, 5 WR","personnelO_0 RB, 1 TE, 4 WR","personnelO_1 RB, 0 TE, 4 WR","personnelO_1 RB, 1 TE, 3 WR","personnelO_1 RB, 2 TE, 2 WR","personnelO_1 RB, 3 TE, 1 WR","personnelO_2 QB, 1 RB, 1 TE, 2 WR","personnelO_2 RB, 0 TE, 3 WR","personnelO_2 RB, 1 TE, 2 WR","personnelO_2 RB, 2 TE, 1 WR","personnelO_3 RB, 1 TE, 1 WR","personnelO_6 OL, 1 RB, 1 TE, 2 WR","personnelO_6 OL, 1 RB, 2 TE, 1 WR","personnelO_6 OL, 2 RB, 1 TE, 0 WR,1 DL","personnelO_6 OL, 2 RB, 2 TE, 0 WR","personnelD_0 DL, 4 LB, 7 DB","personnelD_0 DL, 5 LB, 6 DB","personnelD_0 DL, 6 LB, 5 DB","personnelD_1 DL, 3 LB, 7 DB","personnelD_1 DL, 4 LB, 6 DB","personnelD_1 DL, 5 LB, 5 DB","personnelD_2 DL, 2 LB, 7 DB","personnelD_2 DL, 3 LB, 6 DB","personnelD_2 DL, 4 LB, 5 DB","personnelD_3 DL, 1 LB, 7 DB","personnelD_3 DL, 2 LB, 6 DB","personnelD_3 DL, 3 LB, 5 DB","personnelD_3 DL, 4 LB, 4 DB","personnelD_3 DL, 5 LB, 3 DB","personnelD_4 DL, 1 LB, 6 DB","personnelD_4 DL, 2 LB, 5 DB","personnelD_4 DL, 3 LB, 4 DB","personnelD_4 DL, 4 LB, 3 DB","personnelD_4 DL, 5 LB, 2 DB","personnelD_5 DL, 2 LB, 4 DB",passResult_C,passResult_I,passResult_IN,passResult_S
0,0.00,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,0.00,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,0.00,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,0.59,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
4,0.00,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [15]:
# Set features. This will also be used as your x values.
selected_features = df[['quarter', 'down', 'yardsToGo', 'defendersInTheBox', 'numberOfPassRushers', 'isDefensivePI',
                        'offenseFormation_EMPTY',	'offenseFormation_I_FORM',	'offenseFormation_JUMBO',	'offenseFormation_PISTOL',	'offenseFormation_SHOTGUN',	'offenseFormation_SINGLEBACK',
                        'offenseFormation_WILDCAT',	'personnelO_0 RB, 0 TE, 5 WR',	'personnelO_0 RB, 1 TE, 4 WR',	'personnelO_1 RB, 0 TE, 4 WR',	'personnelO_1 RB, 1 TE, 3 WR',	'personnelO_1 RB, 2 TE, 2 WR',	
                        'personnelO_1 RB, 3 TE, 1 WR',	'personnelO_2 QB, 1 RB, 1 TE, 2 WR',	'personnelO_2 RB, 0 TE, 3 WR',	'personnelO_2 RB, 1 TE, 2 WR',	'personnelO_2 RB, 2 TE, 1 WR',
                        'personnelO_3 RB, 1 TE, 1 WR',	'personnelO_6 OL, 1 RB, 1 TE, 2 WR',	'personnelO_6 OL, 1 RB, 2 TE, 1 WR',	'personnelO_6 OL, 2 RB, 1 TE, 0 WR,1 DL',	'personnelO_6 OL, 2 RB, 2 TE, 0 WR',
                        'personnelD_0 DL, 4 LB, 7 DB',	'personnelD_0 DL, 5 LB, 6 DB',	'personnelD_0 DL, 6 LB, 5 DB',	'personnelD_1 DL, 3 LB, 7 DB',	'personnelD_1 DL, 4 LB, 6 DB',
                        'personnelD_1 DL, 5 LB, 5 DB',	'personnelD_2 DL, 2 LB, 7 DB',	'personnelD_2 DL, 3 LB, 6 DB',	'personnelD_2 DL, 4 LB, 5 DB',	'personnelD_3 DL, 1 LB, 7 DB',
                        'personnelD_3 DL, 2 LB, 6 DB',	'personnelD_3 DL, 3 LB, 5 DB',	'personnelD_3 DL, 4 LB, 4 DB',	'personnelD_3 DL, 5 LB, 3 DB',	'personnelD_4 DL, 1 LB, 6 DB',
                        'personnelD_4 DL, 2 LB, 5 DB',	'personnelD_4 DL, 3 LB, 4 DB',	'personnelD_4 DL, 4 LB, 3 DB',	'personnelD_4 DL, 5 LB, 2 DB',	'personnelD_5 DL, 2 LB, 4 DB'	]]

In [16]:
defensive_features = df[['personnelD_0 DL, 4 LB, 7 DB',	'personnelD_0 DL, 5 LB, 6 DB',	'personnelD_0 DL, 6 LB, 5 DB',	'personnelD_1 DL, 3 LB, 7 DB',	'personnelD_1 DL, 4 LB, 6 DB',
                        'personnelD_1 DL, 5 LB, 5 DB',	'personnelD_2 DL, 2 LB, 7 DB',	'personnelD_2 DL, 3 LB, 6 DB',	'personnelD_2 DL, 4 LB, 5 DB',	'personnelD_3 DL, 1 LB, 7 DB',
                        'personnelD_3 DL, 2 LB, 6 DB',	'personnelD_3 DL, 3 LB, 5 DB',	'personnelD_3 DL, 4 LB, 4 DB',	'personnelD_3 DL, 5 LB, 3 DB',	'personnelD_4 DL, 1 LB, 6 DB',
                        'personnelD_4 DL, 2 LB, 5 DB',	'personnelD_4 DL, 3 LB, 4 DB',	'personnelD_4 DL, 4 LB, 3 DB',	'personnelD_4 DL, 5 LB, 2 DB',	'personnelD_5 DL, 2 LB, 4 DB']]

In [17]:
offensive_features = df[['offenseFormation_EMPTY',	'offenseFormation_I_FORM',	'offenseFormation_JUMBO',	'offenseFormation_PISTOL',	'offenseFormation_SHOTGUN',	'offenseFormation_SINGLEBACK',
                        'offenseFormation_WILDCAT',	'personnelO_0 RB, 0 TE, 5 WR',	'personnelO_0 RB, 1 TE, 4 WR',	'personnelO_1 RB, 0 TE, 4 WR',	'personnelO_1 RB, 1 TE, 3 WR',	'personnelO_1 RB, 2 TE, 2 WR',	
                        'personnelO_1 RB, 3 TE, 1 WR',	'personnelO_2 QB, 1 RB, 1 TE, 2 WR',	'personnelO_2 RB, 0 TE, 3 WR',	'personnelO_2 RB, 1 TE, 2 WR',	'personnelO_2 RB, 2 TE, 1 WR',
                        'personnelO_3 RB, 1 TE, 1 WR',	'personnelO_6 OL, 1 RB, 1 TE, 2 WR',	'personnelO_6 OL, 1 RB, 2 TE, 1 WR',	'personnelO_6 OL, 2 RB, 1 TE, 0 WR,1 DL',	'personnelO_6 OL, 2 RB, 2 TE, 0 WR',]]

**Completion Predictions**

Logistic Regression-----

In [18]:
#import sklearn dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

In [19]:
# logistic regression on complete passes
y = df["passResult_C"]
X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [20]:
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.7214973850812001
Testing Data Score: 0.7213546346984276


In [23]:
import joblib
filename = 'logistic_regression_complete.sav'
joblib.dump(model, filename)

['logistic_regression_complete.sav']

In [24]:
logistic_regression_complete = joblib.load(filename)
logistic_regression_complete

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Random Forest Classifier

In [25]:
# random forest on complete passes
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9779619276148657

In [26]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.22753190202647403, 'yardsToGo'),
 (0.13260708909540966, 'quarter'),
 (0.11784006261999175, 'defendersInTheBox'),
 (0.10807029949073453, 'numberOfPassRushers'),
 (0.0824972397935335, 'down'),
 (0.027395559674165185, 'personnelD_3 DL, 3 LB, 5 DB'),
 (0.026595022380333982, 'personnelD_4 DL, 2 LB, 5 DB'),
 (0.022985548065873215, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.021673302553274293, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.018526595206896434, 'offenseFormation_SHOTGUN'),
 (0.01843196548743937, 'isDefensivePI'),
 (0.0176338415528003, 'personnelO_1 RB, 1 TE, 3 WR'),
 (0.016342765662226297, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.01627035607871528, 'offenseFormation_EMPTY'),
 (0.014011143332421452, 'personnelD_2 DL, 3 LB, 6 DB'),
 (0.01364569584563187, 'offenseFormation_SINGLEBACK'),
 (0.013590297945453382, 'personnelO_1 RB, 2 TE, 2 WR'),
 (0.012251991065114315, 'personnelD_4 DL, 3 LB, 4 DB'),
 (0.0104205601733813, 'personnelO_1 RB, 0 TE, 4 WR'),
 (0.008499959271349737, 'personnelO_2 RB, 1 TE, 2 

In [27]:
filename = 'random_forest_complete.sav'
joblib.dump(rf, filename)

['random_forest_complete.sav']

In [28]:
random_forest_complete = joblib.load(filename)
random_forest_complete

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

**Incomplete Predictions**

Logistic Regression

In [29]:
# logistic regression on incomplete passes
y = df["passResult_I"]
X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [30]:

X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [31]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.7959731869626463
Testing Data Score: 0.7970573805927013


In [33]:
filename = 'logistic_regression_incomplete.sav'
joblib.dump(model, filename)

['logistic_regression_incomplete.sav']

In [34]:
logistic_regression_incomplete = joblib.load(filename)
logistic_regression_incomplete

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Random Forest Classifier

In [35]:
# random forest on incomplete passes
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9910040753675845

In [36]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.24105711261269924, 'yardsToGo'),
 (0.13715480851679196, 'quarter'),
 (0.12342773110263339, 'numberOfPassRushers'),
 (0.10574118680668045, 'defendersInTheBox'),
 (0.07879299349130264, 'down'),
 (0.03272252539241344, 'personnelD_4 DL, 2 LB, 5 DB'),
 (0.025026805956254193, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.02098825566239996, 'personnelD_3 DL, 3 LB, 5 DB'),
 (0.020824098080972308, 'offenseFormation_SHOTGUN'),
 (0.018666900689708534, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.018481763103265304, 'offenseFormation_EMPTY'),
 (0.016734737363846456, 'personnelO_1 RB, 1 TE, 3 WR'),
 (0.015161474234036527, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.013721022521635547, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.012608493240015683, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.012059040662157206, 'personnelO_1 RB, 2 TE, 2 WR'),
 (0.01156663277504459, 'isDefensivePI'),
 (0.011254503681437282, 'personnelD_4 DL, 3 LB, 4 DB'),
 (0.011110697809377859, 'offenseFormation_SINGLEBACK'),
 (0.009527889817294376, 'personnelO_1 RB, 0 T

In [37]:
filename = 'random_forest_complete.sav'
joblib.dump(rf, filename)

['random_forest_complete.sav']

In [38]:
random_forest_complete = joblib.load(filename)
random_forest_complete

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

**Sack Predictions**

Logistic Regression

In [39]:
#logistic regression sacks
y = df["passResult_S"]
X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [40]:
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [41]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [42]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.9300804714949563
Testing Data Score: 0.930009569099096


In [43]:
filename = 'logistic_regression_sacks.sav'
joblib.dump(model, filename)

['logistic_regression_sacks.sav']

In [44]:
logistic_regression_sacks = joblib.load(filename)
logistic_regression_sacks

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Random Forest Clasifier

In [45]:
# random forest on sacks
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9924612985801788

In [46]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.2556203340660554, 'yardsToGo'),
 (0.13350113216979087, 'quarter'),
 (0.12078783679049827, 'defendersInTheBox'),
 (0.11860194157407983, 'numberOfPassRushers'),
 (0.08829884216192267, 'down'),
 (0.03315675614653996, 'personnelD_3 DL, 3 LB, 5 DB'),
 (0.024113962834006575, 'personnelD_4 DL, 2 LB, 5 DB'),
 (0.02048584641338747, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.0202037308960398, 'offenseFormation_SHOTGUN'),
 (0.016250566787421497, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.01574903357451774, 'offenseFormation_SINGLEBACK'),
 (0.015318788991091445, 'personnelO_1 RB, 1 TE, 3 WR'),
 (0.014706447600426711, 'personnelO_2 RB, 1 TE, 2 WR'),
 (0.014525831655857946, 'offenseFormation_I_FORM'),
 (0.012785846279756385, 'offenseFormation_PISTOL'),
 (0.01184417782765786, 'personnelO_1 RB, 2 TE, 2 WR'),
 (0.011662038292817194, 'offenseFormation_EMPTY'),
 (0.010329229801688735, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.010119089530666793, 'personnelD_4 DL, 3 LB, 4 DB'),
 (0.009190442486251218, 'personnelD_1 DL, 5

In [47]:
filename = 'random_forest_sacks.sav'
joblib.dump(rf, filename)

['random_forest_sacks.sav']

In [48]:
random_forest_sacks = joblib.load(filename)
random_forest_sacks

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

**Interceptions Predictions**

Logistic Regression

In [49]:
#logistic regression on interceptions
y = df["passResult_IN"]
X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [50]:
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [51]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [52]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.9646637845889801
Testing Data Score: 0.9644826128983683


In [53]:
filename = 'logistic_regression_interceptions.sav'
joblib.dump(model, filename)

['logistic_regression_interceptions.sav']

In [54]:
logistic_regression_interceptions = joblib.load(filename)
logistic_regression_interceptions

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Random Forest Classifier

In [55]:
#random forest on interceptions
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9946568482204876

In [56]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.15720293386711337, 'numberOfPassRushers'),
 (0.15092370373364725, 'quarter'),
 (0.12517437281464283, 'yardsToGo'),
 (0.10282808824940531, 'defendersInTheBox'),
 (0.08734917701962312, 'down'),
 (0.04128440604595754, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.029042264210329672, 'offenseFormation_SHOTGUN'),
 (0.02667893727426899, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.022626449024520094, 'personnelD_3 DL, 3 LB, 5 DB'),
 (0.02225384880174768, 'offenseFormation_WILDCAT'),
 (0.021957269410270697, 'offenseFormation_EMPTY'),
 (0.021459210241541803, 'personnelD_4 DL, 2 LB, 5 DB'),
 (0.01791655888753975, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.016094082168753285, 'offenseFormation_PISTOL'),
 (0.015824799784561437, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.014724103496388842, 'personnelD_2 DL, 3 LB, 6 DB'),
 (0.014408982896821455, 'personnelO_1 RB, 2 TE, 2 WR'),
 (0.014175348380444722, 'offenseFormation_I_FORM'),
 (0.012710349804422877, 'personnelO_2 RB, 1 TE, 2 WR'),
 (0.012177221523475328, 'personnelO_1 RB, 0

In [57]:
filename = 'random_forest_interceptions.sav'
joblib.dump(rf, filename)

['random_forest_interceptions.sav']

In [58]:
random_forest_interceptions = joblib.load(filename)
random_forest_interceptions

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

**Catch Predictions vs Defensive formations**

Logistic Regression

In [59]:
#logistic regression complete passes vs defensive formations
y = df["passResult_C"]
X = defensive_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [60]:
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [61]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [62]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.6863975648062693
Testing Data Score: 0.6853466491152227


In [63]:
filename = 'logistic_regression_complete_defense.sav'
joblib.dump(model, filename)

['logistic_regression_complete_defense.sav']

In [64]:
logistic_regression_complete_defense = joblib.load(filename)
logistic_regression_complete_defense

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Random Forest Classifier

In [65]:
#random forest complete passes vs defensive formations
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.6853466491152227

In [66]:
sorted(zip(rf.feature_importances_, defensive_features), reverse=True)

[(0.18074132173600946, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.1683952904388161, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.15362066924737894, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.06508649117166677, 'personnelD_2 DL, 3 LB, 6 DB'),
 (0.05463192659963242, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.05424246783548972, 'personnelD_0 DL, 5 LB, 6 DB'),
 (0.04724539990229405, 'personnelD_4 DL, 3 LB, 4 DB'),
 (0.03882528092707768, 'personnelD_4 DL, 5 LB, 2 DB'),
 (0.037333399048226884, 'personnelD_3 DL, 4 LB, 4 DB'),
 (0.0366729924942544, 'personnelD_3 DL, 3 LB, 5 DB'),
 (0.03142138155917715, 'personnelD_4 DL, 4 LB, 3 DB'),
 (0.02722016797442695, 'personnelD_1 DL, 5 LB, 5 DB'),
 (0.020346988984362577, 'personnelD_5 DL, 2 LB, 4 DB'),
 (0.017505177458079, 'personnelD_4 DL, 2 LB, 5 DB'),
 (0.015840216297463027, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.015320568779529996, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.013872096186836273, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.010697646507180275, 'personnelD_0 DL, 4 LB, 7 DB'),
 (0.0097

In [67]:
filename = 'random_forest_complete_defense.sav'
joblib.dump(rf, filename)

['random_forest_complete_defense.sav']

In [68]:
random_forest_complete_defense = joblib.load(filename)
random_forest_complete_defense

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

**Incomplete Passes vs Offense formations**

Logistic Regression

In [69]:
#logistic regression incomplete passes vs offense formations
y = df["passResult_I"]
X = offensive_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [70]:
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [71]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [72]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.7736824209452567
Testing Data Score: 0.7738292425839483


In [73]:
filename = 'logistic_regression_incomplete_offense.sav'
joblib.dump(model, filename)

['logistic_regression_incomplete_offense.sav']

In [74]:
logistic_regression_incomplete_offense = joblib.load(filename)
logistic_regression_incomplete_offense

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Random Forest Classifier

In [75]:
#random forest incomplete passes vs offense formations
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.7765931092771687

In [76]:
sorted(zip(rf.feature_importances_, offensive_features), reverse=True)

[(0.1402448337507129, 'offenseFormation_SINGLEBACK'),
 (0.11196974065847118, 'offenseFormation_SHOTGUN'),
 (0.1008119498246258, 'personnelO_1 RB, 1 TE, 3 WR'),
 (0.09817224340191293, 'personnelO_1 RB, 0 TE, 4 WR'),
 (0.09188933881527359, 'offenseFormation_EMPTY'),
 (0.08264580997964728, 'personnelO_2 RB, 0 TE, 3 WR'),
 (0.07157198027147832, 'personnelO_0 RB, 1 TE, 4 WR'),
 (0.04749336761878835, 'offenseFormation_PISTOL'),
 (0.04544833859329457, 'personnelO_2 RB, 1 TE, 2 WR'),
 (0.04331536691767728, 'personnelO_1 RB, 2 TE, 2 WR'),
 (0.038573532436418874, 'offenseFormation_I_FORM'),
 (0.031120124646091443, 'personnelO_1 RB, 3 TE, 1 WR'),
 (0.02925591735931512, 'personnelO_6 OL, 2 RB, 2 TE, 0 WR'),
 (0.01814485547870588, 'personnelO_2 QB, 1 RB, 1 TE, 2 WR'),
 (0.010679473055164946, 'personnelO_3 RB, 1 TE, 1 WR'),
 (0.010678143443680087, 'personnelO_2 RB, 2 TE, 1 WR'),
 (0.0064978035504976925, 'offenseFormation_JUMBO'),
 (0.005763179323655934, 'personnelO_6 OL, 2 RB, 1 TE, 0 WR,1 DL'),
 (0

In [77]:
filename = 'random_forest_incomplete_offense.sav'
joblib.dump(rf, filename)

['random_forest_incomplete_offense.sav']

In [78]:
random_forest_incomplete_offense = joblib.load(filename)
random_forest_incomplete_offense

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

**Exta DB's and Incomplete Catches**

Logistic Regression

In [79]:
moreDB_features = df[['personnelD_0 DL, 4 LB, 7 DB',	'personnelD_0 DL, 5 LB, 6 DB',	'personnelD_0 DL, 6 LB, 5 DB',	'personnelD_1 DL, 3 LB, 7 DB',	
                      'personnelD_1 DL, 4 LB, 6 DB', 'personnelD_1 DL, 5 LB, 5 DB',	'personnelD_2 DL, 2 LB, 7 DB', 'personnelD_2 DL, 4 LB, 5 DB',	
                      'personnelD_3 DL, 1 LB, 7 DB', 'personnelD_3 DL, 2 LB, 6 DB', 'personnelD_4 DL, 1 LB, 6 DB']]

In [80]:
#logistic regression extra db's vs incomplete passes
y = df["passResult_I"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [81]:
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [82]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [83]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.7872427583750263
Testing Data Score: 0.7872891276576108


In [84]:
filename = 'logistic_regression_extra_incomplete.sav'
joblib.dump(model, filename)

['logistic_regression_extra_incomplete.sav']

In [85]:
logistic_regression_extra_incomplete = joblib.load(filename)
logistic_regression_extra_incomplete

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Random Forest Classifier

In [86]:
#random forest extra db's vs incomplete passes
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.7872891276576108

In [87]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

[(0.30831525060928855, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.2781381263993078, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.24695905831514048, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.09983969312041979, 'personnelD_0 DL, 5 LB, 6 DB'),
 (0.03885855488213396, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.008286542420264465, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.007268731981565456, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.006516186867247729, 'personnelD_3 DL, 1 LB, 7 DB'),
 (0.0053870796454815375, 'personnelD_0 DL, 4 LB, 7 DB'),
 (0.0003173383131327562, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.00011343744601744187, 'personnelD_1 DL, 5 LB, 5 DB')]

In [88]:
filename = 'random_forest_extra_incomplete.sav'
joblib.dump(rf, filename)

['random_forest_extra_incomplete.sav']

In [89]:
logistic_regression_extra_incomplete = joblib.load(filename)
logistic_regression_extra_incomplete

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

**Extra DB and Interceptions**

Logistic Regression

In [90]:
#logistic regression extra db's vs interceptions
y = df["passResult_IN"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [91]:
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [92]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [93]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.9621136315797996
Testing Data Score: 0.9620539075440445


In [94]:
filename = 'logistic_regression_extra_interceptions.sav'
joblib.dump(model, filename) 

['logistic_regression_extra_interceptions.sav']

In [95]:
logistic_regression_extra_interceptions = joblib.load(filename)
logistic_regression_extra_interceptions

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Random Forest Classifier

In [96]:
#random forest extra db's vs interceptions
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9620539075440445

In [97]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

[(0.9110774571729607, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.027966051199638034, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.01505741287583008, 'personnelD_1 DL, 5 LB, 5 DB'),
 (0.013100820065924008, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.013065721207087972, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.008122256039301928, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.0038093908494472375, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.003252752867005136, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.0026391661623257506, 'personnelD_3 DL, 1 LB, 7 DB'),
 (0.0010976047122883592, 'personnelD_0 DL, 5 LB, 6 DB'),
 (0.000811366848190776, 'personnelD_0 DL, 4 LB, 7 DB')]

In [98]:
filename = 'random_forest_extra_interceptions.sav'
joblib.dump(rf, filename) 

['random_forest_extra_interceptions.sav']

In [99]:
random_forest_extra_interceptions = joblib.load(filename)
random_forest_extra_interceptions

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

**Extra DB and Sacks**

Logistic Regression

In [100]:
#logistic regression extra db's vs sacks
y = df["passResult_S"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [101]:
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [102]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [103]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.9290425996988391
Testing Data Score: 0.9290429443680751


In [104]:
filename = 'logistic_regression_extra_sacks.sav'
joblib.dump(model, filename) 

['logistic_regression_extra_sacks.sav']

In [105]:
logistic_regression_extra_sacks = joblib.load(filename)
logistic_regression_extra_sacks

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Random Forest

In [106]:
#random forest extra db's vs sacks
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9290429443680751

In [107]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

[(0.37416992023734397, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.23953967577065177, 'personnelD_1 DL, 5 LB, 5 DB'),
 (0.17714449024238574, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.10812240673054659, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.028233437839711117, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.02473292282035846, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.02034625932414251, 'personnelD_3 DL, 1 LB, 7 DB'),
 (0.00897981941645065, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.008550999974695531, 'personnelD_0 DL, 5 LB, 6 DB'),
 (0.006866425481918299, 'personnelD_0 DL, 4 LB, 7 DB'),
 (0.0033136421617953214, 'personnelD_4 DL, 1 LB, 6 DB')]

In [108]:
filename = 'random_forest_extra_sacks.sav'
joblib.dump(rf, filename) 

['random_forest_extra_sacks.sav']

In [109]:
random_forest_extra_sacks = joblib.load(filename)
random_forest_extra_sacks

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

**Extra DB's and Catches**

Logistic Regression

In [110]:
#logistic regression extra db's vs completions
y = df["passResult_C"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [111]:
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [112]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [113]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.6844157316105633
Testing Data Score: 0.6834814034031019


In [114]:
filename = 'logistic_regression_extra_complete.sav'
joblib.dump(model, filename) 

['logistic_regression_extra_complete.sav']

In [115]:
logistic_regression_extra_complete = joblib.load(filename)
logistic_regression_extra_complete

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Random Forest Classifier

In [116]:
#random forest extra db's vs completions
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.6834814034031019

In [117]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

[(0.28004531531410065, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.2430217543762155, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.2215138594418018, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.08039145476756257, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.07857982323364493, 'personnelD_0 DL, 5 LB, 6 DB'),
 (0.04265555722583594, 'personnelD_1 DL, 5 LB, 5 DB'),
 (0.025206841221830274, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.012932585048679258, 'personnelD_0 DL, 4 LB, 7 DB'),
 (0.007639503899539843, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.007463622622950172, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.0005496828478391294, 'personnelD_3 DL, 1 LB, 7 DB')]

In [118]:
filename = 'random_forest_extra_complete.sav'
joblib.dump(rf, filename) 

['random_forest_extra_complete.sav']

In [119]:
random_forest_extra_complete = joblib.load(filename)
random_forest_extra_complete

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

**Offensive Formation agaisnt Extra DB's**

Logistic Regression

In [120]:
#logistic regression offense formation vs extra dbs
y = df["offenseFormation_I_FORM"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [121]:
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [122]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [123]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.9533475818072894
Testing Data Score: 0.953349427554148


In [124]:
filename = 'logistic_regression_offense_extra.sav'
joblib.dump(model, filename) 

['logistic_regression_offense_extra.sav']

In [125]:
logistic_regression_offense_extra = joblib.load(filename)
logistic_regression_offense_extra

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Random Forest

In [126]:
#random forest offense formation vs extra dbs
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.953349427554148

In [127]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

[(0.285191645568188, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.27878953604967854, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.21801699659143037, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.0900305295088272, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.027780794296605328, 'personnelD_1 DL, 5 LB, 5 DB'),
 (0.027485105338947297, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.024529459433560732, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.02131793324332357, 'personnelD_3 DL, 1 LB, 7 DB'),
 (0.009609362899696964, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.009546488831946049, 'personnelD_0 DL, 5 LB, 6 DB'),
 (0.007702148237795934, 'personnelD_0 DL, 4 LB, 7 DB')]

In [128]:
filename = 'random_forest_offense_extra.sav'
joblib.dump(rf, filename) 

['random_forest_offense_extra.sav']

In [129]:
random_forest_offense_extra = joblib.load(filename)
random_forest_offense_extra

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)